# Imports

In [59]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import classification_report
from xgboost import XGBClassifier

In [65]:
pd.set_option('display.max_columns', 500)

# Load data

In [3]:
data = pd.read_csv('../data/clickdata.csv')

data.head()

,epoch_ms,session_id,country_by_ip_address,region_by_ip_address,url_without_parameters,referrer_without_parameters,visitor_recognition_type,ua_agent_class
0,1520280001034,be73c8d1b836170a21529a1b23140f8e,US,CA,https://www.bol.com/nl/l/nederlandstalige-kuns...,NaN,ANONYMOUS,Robot
1,1520280001590,c24c6637ed7dcbe19ad64056184212a7,US,CA,https://www.bol.com/nl/l/italiaans-natuur-wete...,NaN,ANONYMOUS,Robot
2,1520280002397,ee391655f5680a7bfae0019450aed396,IT,LI,https://www.bol.com/nl/p/nespresso-magimix-ini...,https://www.bol.com/nl/p/nespresso-magimix-ini...,ANONYMOUS,Browser
3,1520280002598,f8c8a696dd37ca88233b2df096afa97f,US,CA,https://www.bol.com/nl/l/nieuwe-engelstalige-o...,NaN,ANONYMOUS,Robot
4,1520280004428,f8b0c06747b7dd1d53c0932306bd04d6,US,CA,https://www.bol.com/nl/l/nieuwe-actie-avontuur...,NaN,ANONYMOUS,Robot Mobile


## Preprocess/create features

In [42]:
# Add stratification label
data['stratification'] = data['visitor_recognition_type'] + '_' + data['ua_agent_class']

# Filter out labels with less than min samples
data = data.loc[~data['stratification'].str.contains('RECOGNIZED_Hacker'), :]
data = data.loc[~data['ua_agent_class'].isin(['Cloud Application', 'Mobile App']), :]

# Filling in missing values
data.loc[data['country_by_ip_address'].isna(), 'country_by_ip_address'] = 'UNK'
data.loc[data['region_by_ip_address'].isna(), 'region_by_ip_address'] = 'UNK'
data.loc[data['referrer_without_parameters'].isna(), 'referrer_without_parameters'] = ''

# Splitting class into class and source
data.loc[data['ua_agent_class'] == 'Browser Webview', 'ua_source'] = 'Webview'
data.loc[data['ua_agent_class'] == 'Browser Webview', 'ua_agent_class'] = 'Browser'
data.loc[data['ua_agent_class'] == 'Robot Mobile', 'ua_source'] = 'Mobile'
data.loc[data['ua_agent_class'] == 'Robot Mobile', 'ua_agent_class'] = 'Robot'

data.head()

,epoch_ms,session_id,country_by_ip_address,region_by_ip_address,url_without_parameters,referrer_without_parameters,visitor_recognition_type,ua_agent_class,ua_source,url_function,...,attribute_filters,n_attribute_filters,search_type,search_text,search_context,Nty,product_id,other,tracking_id,stratification
0,1520280001034,be73c8d1b836170a21529a1b23140f8e,US,CA,https://www.bol.com/nl/l/nederlandstalige-kuns...,,ANONYMOUS,Robot,NaN,l,...,[],0,,,,,,,,ANONYMOUS_Robot
1,1520280001590,c24c6637ed7dcbe19ad64056184212a7,US,CA,https://www.bol.com/nl/l/italiaans-natuur-wete...,,ANONYMOUS,Robot,NaN,l,...,[],0,,,,,,,,ANONYMOUS_Robot
2,1520280002397,ee391655f5680a7bfae0019450aed396,IT,LI,https://www.bol.com/nl/p/nespresso-magimix-ini...,https://www.bol.com/nl/p/nespresso-magimix-ini...,ANONYMOUS,Browser,NaN,p,...,[],0,,,,,9200000025533140,,,ANONYMOUS_Browser
3,1520280002598,f8c8a696dd37ca88233b2df096afa97f,US,CA,https://www.bol.com/nl/l/nieuwe-engelstalige-o...,,ANONYMOUS,Robot,NaN,l,...,[4273962351],1,,,,,,,,ANONYMOUS_Robot
4,1520280004428,f8b0c06747b7dd1d53c0932306bd04d6,US,CA,https://www.bol.com/nl/l/nieuwe-actie-avontuur...,,ANONYMOUS,Robot,Mobile,l,...,[],0,,,,,,,,ANONYMOUS_Robot


## Parse URLs

In [45]:
def parse_url(url, prefix = 'https://www.bol.com/nl/'):
    def is_product_code(x):
        pass

    url_components = url.removeprefix(prefix).split('/')
    row = {
        'url_function': '',
        'category': '',
        'category_id': '',
        'category_filters': [],
        'n_category_filters': 0,
        'attribute_filters': [],
        'n_attribute_filters': 0,
        'search_type': '',
        'search_text': '',
        'search_context': '',
        'Nty': '',
        'product_id': '',
        'other': '',
        'tracking_id': ''
    }

    if url_components[0] == 'c':
        row['url_function'] = url_components[0]

        if url_components[1] == 'ajax':
            row['other'] = url_components[1]

        else:
            row['category'] = url_components[1]

            if url_components[2].isdigit():
                row['category_id'] = url_components[2]

            else:
                row['category'] = row['category'] + '/' + url_components[2]

                if url_components[3].isdigit():
                    row['category_id'] = url_components[3]

        if 'N' in url_components:
            index = url_components.index('N')
            row['category_filters'] = url_components[index + 1].split('+')

        if 'sc' in url_components:
            index = url_components.index('sc')
            row['search_context'] = url_components[index + 1]

        if 'filter_N' in url_components:
            index = url_components.index('filter_N')
            row['attribute_filters'] = url_components[index + 1].split('+')

    elif url_components[0] == 'checkout':
        row['url_function'] = url_components[0]
        row['other'] = url_components[1]

    elif url_components[0] == 'l':
        row['url_function'] = url_components[0]

        if url_components[1] == 'ajax':
            row['other'] = url_components[1]

        else:
            row['category'] = url_components[1]

        if 'N' in url_components:
            index = url_components.index('N')
            row['category_filters'] = url_components[index + 1].split('+')

        if 'filter_N' in url_components:
            index = url_components.index('filter_N')
            row['attribute_filters'] = url_components[index + 1].split('+')

    elif url_components[0] == 'order':
        row['url_function'] = url_components[0]
        row['other'] = url_components[1]

    elif url_components[0] == 'p':
        row['url_function'] = url_components[0]
        row['category'] = url_components[1]

        if url_components[2].isdigit():
            row['product_id'] = url_components[2]

    elif url_components[0] == 's':
        row['url_function'] = url_components[0]

        if url_components[1].isdigit():
            row['category_id'] = url_components[1]
        
        else:
            row['category'] = url_components[1]

        if 'N' in url_components:
            index = url_components.index('N')
            row['category_filters'] = url_components[index + 1].split('+')

        if 'Ntt' in url_components:
            index = url_components.index('Ntt')
            row['search_text'] = url_components[index + 1]

        if 'Nty' in url_components:
            index = url_components.index('Nty')
            row['Nty'] = url_components[index + 1]

        if 'sc' in url_components:
            index = url_components.index('sc')
            row['search_context'] = url_components[index + 1]

        if 'filter_N' in url_components:
            index = url_components.index('filter_N')
            row['attribute_filters'] = url_components[index + 1].split('+')

        if 'ajax' in url_components:
            row['other'] = 'ajax'

    elif url_components[0] == 'w':
        row['url_function'] = url_components[0]

        if url_components[1] == 'ajax':
            row['other'] = url_components[1]

        else:
            row['category'] = url_components[1]

            if url_components[2].isdigit():
                row['tracking_id'] = url_components[2]

            else:
                row['category'] = row['category'] + '/' + url_components[2]
                row['tracking_id'] = url_components[3]

            if 'N' in url_components:
                index = url_components.index('N')
                row['category_filters'] = url_components[index + 1].split('+')

            if 'filter_N' in url_components:
                index = url_components.index('filter_N')
                row['attribute_filters'] = url_components[index + 1].split('+')              

    row['n_category_filters'] = len(row['category_filters'])
    row['n_attribute_filters'] = len(row['attribute_filters'])

    return pd.Series(row)

In [46]:
url_features = ['url_function',
                'category',
                'category_id',
                'category_filters',
                'n_category_filters',
                'attribute_filters',
                'n_attribute_filters',
                'search_type',
                'search_text',
                'search_context',
                'Nty',
                'product_id',
                'other',
                'tracking_id']

data[url_features] = data['url_without_parameters'].apply(lambda url: parse_url(url))

data.head()

,epoch_ms,session_id,country_by_ip_address,region_by_ip_address,url_without_parameters,referrer_without_parameters,visitor_recognition_type,ua_agent_class,ua_source,url_function,...,attribute_filters,n_attribute_filters,search_type,search_text,search_context,Nty,product_id,other,tracking_id,stratification
0,1520280001034,be73c8d1b836170a21529a1b23140f8e,US,CA,https://www.bol.com/nl/l/nederlandstalige-kuns...,,ANONYMOUS,Robot,NaN,l,...,[],0,,,,,,,,ANONYMOUS_Robot
1,1520280001590,c24c6637ed7dcbe19ad64056184212a7,US,CA,https://www.bol.com/nl/l/italiaans-natuur-wete...,,ANONYMOUS,Robot,NaN,l,...,[],0,,,,,,,,ANONYMOUS_Robot
2,1520280002397,ee391655f5680a7bfae0019450aed396,IT,LI,https://www.bol.com/nl/p/nespresso-magimix-ini...,https://www.bol.com/nl/p/nespresso-magimix-ini...,ANONYMOUS,Browser,NaN,p,...,[],0,,,,,9200000025533140,,,ANONYMOUS_Browser
3,1520280002598,f8c8a696dd37ca88233b2df096afa97f,US,CA,https://www.bol.com/nl/l/nieuwe-engelstalige-o...,,ANONYMOUS,Robot,NaN,l,...,[4273962351],1,,,,,,,,ANONYMOUS_Robot
4,1520280004428,f8b0c06747b7dd1d53c0932306bd04d6,US,CA,https://www.bol.com/nl/l/nieuwe-actie-avontuur...,,ANONYMOUS,Robot,Mobile,l,...,[],0,,,,,,,,ANONYMOUS_Robot


# Base model

In [11]:
X = data[['country_by_ip_address', 'region_by_ip_address', 'visitor_recognition_type']].astype('category')

le = LabelEncoder()
y = le.fit_transform(data['ua_agent_class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf = XGBClassifier(tree_method="hist", enable_categorical=True)

clf.fit(X_train, y_train, verbose=False)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [16]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0.0))

                   precision    recall  f1-score   support

          Browser       0.98      0.99      0.99      9328
Cloud Application       0.00      0.00      0.00         1
           Hacker       0.00      0.00      0.00       294
       Mobile App       0.00      0.00      0.00         2
            Robot       0.95      1.00      0.97      5285
          Special       0.67      0.06      0.10        36

         accuracy                           0.97     14946
        macro avg       0.43      0.34      0.34     14946
     weighted avg       0.95      0.97      0.96     14946



## With url_function

In [47]:
X = data[['country_by_ip_address', 'region_by_ip_address', 'visitor_recognition_type', 'url_function']].astype('category')

le = LabelEncoder()
y = le.fit_transform(data['ua_agent_class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf = XGBClassifier(tree_method="hist", enable_categorical=True)

clf.fit(X_train, y_train, verbose=False)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [48]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0.0))

              precision    recall  f1-score   support

     Browser       0.98      0.99      0.99      9328
      Hacker       0.65      0.04      0.08       294
       Robot       0.96      1.00      0.98      5285
     Special       0.67      0.06      0.10        36

    accuracy                           0.97     14943
   macro avg       0.81      0.52      0.54     14943
weighted avg       0.97      0.97      0.96     14943



# With URL components

In [23]:
features = ['country_by_ip_address', 
            'region_by_ip_address', 
            'visitor_recognition_type',
            'url_function',
            'category',
            'category_id',
            'n_category_filters',
            'n_attribute_filters',
            'search_type',
            'search_text',
            'search_context',
            'Nty',
            'product_id',
            'other',
            'tracking_id']

cat_features = ['country_by_ip_address', 
                'region_by_ip_address', 
                'visitor_recognition_type',
                'url_function',
                'category',
                'category_id',
                'search_type',
                'search_text',
                'search_context',
                'Nty',
                'product_id',
                'other',
                'tracking_id']

numerical_features = ['n_category_filters',
                        'n_attribute_filters']
                        
X = pd.concat([data[numerical_features],
                data[cat_features].astype('category')], axis=1)

le = LabelEncoder()
y = le.fit_transform(data['ua_agent_class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

clf = XGBClassifier(tree_method="hist", enable_categorical=True)

clf.fit(X_train, y_train, verbose=False)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [24]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0.0))

                   precision    recall  f1-score   support

          Browser       0.99      0.99      0.99      9328
Cloud Application       0.00      0.00      0.00         1
           Hacker       0.91      0.55      0.69       294
       Mobile App       0.00      0.00      0.00         2
            Robot       0.96      0.99      0.98      5285
          Special       0.67      0.06      0.10        36

         accuracy                           0.98     14946
        macro avg       0.59      0.43      0.46     14946
     weighted avg       0.98      0.98      0.98     14946



# With multi column stratification

In [ ]:
features = ['country_by_ip_address', 
            'region_by_ip_address', 
            'visitor_recognition_type',
            'url_function',
            'category',
            'category_id',
            'n_category_filters',
            'n_attribute_filters',
            'search_type',
            'search_text',
            'search_context',
            'Nty',
            'product_id',
            'other',
            'tracking_id']

cat_features = ['country_by_ip_address', 
                'region_by_ip_address', 
                'visitor_recognition_type',
                'url_function',
                'category',
                'category_id',
                'search_type',
                'search_text',
                'search_context',
                'Nty',
                'product_id',
                'other',
                'tracking_id']

numerical_features = ['n_category_filters',
                        'n_attribute_filters']



X = pd.concat([data[numerical_features],
                data[cat_features].astype('category')], axis=1)

le = LabelEncoder()
y = le.fit_transform(data['ua_agent_class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=data['stratification'], random_state=42)

clf = XGBClassifier(tree_method="hist", enable_categorical=True)

clf.fit(X_train, y_train, verbose=False)

,objective,'multi:softprob'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [33]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0.0))

              precision    recall  f1-score   support

     Browser       1.00      0.99      0.99      9328
      Hacker       0.89      0.61      0.72       294
  Mobile App       0.00      0.00      0.00         2
       Robot       0.97      0.99      0.98      5285
     Special       0.60      0.17      0.26        36

    accuracy                           0.98     14945
   macro avg       0.69      0.55      0.59     14945
weighted avg       0.98      0.98      0.98     14945



# Hyperparameter tuned

In [ ]:
features = ['country_by_ip_address', 
            'region_by_ip_address', 
            'visitor_recognition_type',
            'url_function',
            'category',
            'category_id',
            'n_category_filters',
            'n_attribute_filters',
            'search_type',
            'search_text',
            'search_context',
            'Nty',
            'product_id',
            'other',
            'tracking_id']

cat_features = ['country_by_ip_address', 
                'region_by_ip_address', 
                'visitor_recognition_type',
                'url_function',
                'category',
                'category_id',
                'search_type',
                'search_text',
                'search_context',
                'Nty',
                'product_id',
                'other',
                'tracking_id']

numerical_features = ['n_category_filters',
                        'n_attribute_filters']

X = pd.concat([data[numerical_features],
                data[cat_features].astype('category')], axis=1)

le = LabelEncoder()
y = le.fit_transform(data['ua_agent_class'])

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=data['stratification'], random_state=42)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Hyperparameter search space
param_dist = {
    # Learning rate and trees
    # "n_estimators": np.arange(100, 1100, 100),
    # "learning_rate": [0.01, 0.05, 0.1, 0.2],

    # Tree structure
    "max_depth": np.arange(2, 12, 2),
    # "min_child_weight": np.arange(1, 10, 2),

    # # Sampling
    # "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    # "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],

    # # Regularization
    # "gamma": [0, 0.1, 0.2, 0.3, 0.4],
    # "reg_alpha": [0, 0.01, 0.05, 0.1, 1, 10],  # L1
    # "reg_lambda": [0.1, 0.5, 1, 5, 10],       # L2
}

# TODO: Causes an error. Needs custom implementation
# Required for stratification on multi-column ua_agent_class + visitor_recognition_type
# cv_splitter = skf.split(X, data['stratification'])

scoring = {
    "f1_macro": make_scorer(f1_score, average="macro"),
    "f1_weighted": make_scorer(f1_score, average="weighted")
}

clf = XGBClassifier(tree_method="hist", enable_categorical=True)

search = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param_dist,
    n_iter=100,
    scoring=scoring,
    refit='f1_macro',
    cv=skf,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

search_results = search.fit(X_train, y_train)

InvalidParameterError: The 'cv' parameter of RandomizedSearchCV must be an int in the range [2, inf), an object implementing 'split' and 'get_n_splits', an iterable or None. Got <function cv_splitter at 0x122697eb0> instead.

## max_depth

In [67]:
pd.DataFrame(search.cv_results_).sort_values('rank_test_f1_macro')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,params,split0_test_f1_macro,split1_test_f1_macro,split2_test_f1_macro,split3_test_f1_macro,split4_test_f1_macro,mean_test_f1_macro,std_test_f1_macro,rank_test_f1_macro,split0_test_f1_weighted,split1_test_f1_weighted,split2_test_f1_weighted,split3_test_f1_weighted,split4_test_f1_weighted,mean_test_f1_weighted,std_test_f1_weighted,rank_test_f1_weighted
4,83.865474,8.984957,0.405161,0.019027,10,{'max_depth': 10},0.726288,0.708997,0.710215,0.734017,0.685384,0.712980,0.016761,1,0.980967,0.980531,0.980151,0.981190,0.978923,0.980353,0.000800,1
1,32.489500,0.263402,0.347845,0.017871,4,{'max_depth': 4},0.728265,0.709782,0.706774,0.733456,0.680509,0.711757,0.018700,2,0.980821,0.980704,0.979834,0.981029,0.978278,0.980133,0.001013,3
2,59.031283,1.570378,0.552050,0.016796,6,{'max_depth': 6},0.727495,0.708462,0.707029,0.730911,0.680631,0.710906,0.017961,3,0.981218,0.980493,0.980233,0.980683,0.978500,0.980225,0.000921,2
3,82.567876,2.618434,0.490690,0.037374,8,{'max_depth': 8},0.725577,0.707237,0.708777,0.730374,0.680591,0.710511,0.017495,4,0.980752,0.980395,0.980258,0.980644,0.978395,0.980089,0.000865,4
0,13.594531,0.075898,0.194880,0.007540,2,{'max_depth': 2},0.723788,0.703167,0.706281,0.727641,0.678109,0.707797,0.017633,5,0.980474,0.979780,0.979732,0.979992,0.979063,0.979808,0.000456,5


## learning_rate

In [69]:
pd.DataFrame(search.cv_results_).sort_values('rank_test_f1_macro')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,params,split0_test_f1_macro,split1_test_f1_macro,split2_test_f1_macro,split3_test_f1_macro,split4_test_f1_macro,mean_test_f1_macro,std_test_f1_macro,rank_test_f1_macro,split0_test_f1_weighted,split1_test_f1_weighted,split2_test_f1_weighted,split3_test_f1_weighted,split4_test_f1_weighted,mean_test_f1_weighted,std_test_f1_weighted,rank_test_f1_weighted
1,78.293679,1.486478,0.661131,0.056568,0.05,{'learning_rate': 0.05},0.731971,0.702519,0.706925,0.743395,0.673100,0.711582,0.024546,1,0.981114,0.979625,0.980013,0.979832,0.978088,0.979735,0.000971,3
3,54.824305,1.071503,0.454280,0.056354,0.20,{'learning_rate': 0.2},0.729580,0.708998,0.708012,0.731638,0.678883,0.711422,0.019051,2,0.981430,0.980378,0.980089,0.980901,0.978370,0.980234,0.001038,1
2,63.546699,2.731138,0.508948,0.150976,0.10,{'learning_rate': 0.1},0.732471,0.703144,0.708702,0.730575,0.675763,0.710131,0.020732,3,0.981412,0.979825,0.980307,0.980656,0.977862,0.980012,0.001193,2
0,80.124923,1.794696,0.597218,0.062401,0.01,{'learning_rate': 0.01},0.709914,0.692233,0.707281,0.740723,0.666175,0.703265,0.024334,4,0.978352,0.978520,0.979720,0.979201,0.976904,0.978540,0.000953,4


## min_child_weight

In [72]:
pd.DataFrame(search.cv_results_).sort_values('rank_test_f1_macro')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_child_weight,params,split0_test_f1_macro,split1_test_f1_macro,split2_test_f1_macro,split3_test_f1_macro,split4_test_f1_macro,mean_test_f1_macro,std_test_f1_macro,rank_test_f1_macro,split0_test_f1_weighted,split1_test_f1_weighted,split2_test_f1_weighted,split3_test_f1_weighted,split4_test_f1_weighted,mean_test_f1_weighted,std_test_f1_weighted,rank_test_f1_weighted
1,711.163836,501.372242,204.644803,407.680173,3,{'min_child_weight': 3},0.731138,0.709082,0.707410,0.732493,0.681689,0.712362,0.018622,1,0.980945,0.980337,0.979765,0.980635,0.978318,0.980000,0.000927,2
2,1097.839268,820.936903,0.905081,0.318804,5,{'min_child_weight': 5},0.731154,0.710877,0.706689,0.731242,0.681687,0.712330,0.018357,2,0.980943,0.980475,0.979600,0.980126,0.978317,0.979892,0.000902,3
0,1124.565283,1.049437,0.439824,0.047635,1,{'min_child_weight': 1},0.727495,0.708462,0.707029,0.730911,0.680631,0.710906,0.017961,3,0.981218,0.980493,0.980233,0.980683,0.978500,0.980225,0.000921,1
3,743.102630,3.096545,0.897183,0.089962,7,{'min_child_weight': 7},0.732523,0.711552,0.687759,0.727634,0.678628,0.707619,0.021313,4,0.981263,0.980581,0.979549,0.979690,0.977866,0.979790,0.001146,4
